In [9]:
#===========================================
# import modules, defs and variables
#===========================================
exec(open("./MSI-generator-def.py").read())

print('Finish modules, defs and variables import, next step: check the dimensionality of this MSI data set.')

Finish modules, defs and variables import, next step: check the dimensionality of this MSI data set.


### configurations

In [2]:
# raw file path without the line number or extension
NameBody =  r".\data\data-file-line"    # edit
# file extension
NamePost = ".d"                                                           # edit

# specify mass list file dir
MassList_dir = r".\data\QTOF-mass-list.xlsx"                    # edit

# mass window (ppm)
MassTolerance = 10

# Locate the extraction output folder. 
MSI_data_output = r".\output"                   # edit

In [3]:
# import mass list
# Mass list should be an excel sheet with a header "m/z"
df_MassList = pd.read_excel(MassList_dir)
MassList = df_MassList['m/z'].values

# get a list of all raw files
LineList = Get_LineList(NameBody, NamePost)

# check dimensions for all line scans
NumSpePerLine_unaligned, tot_acq_times = check_dim(LineList, NameBody, NamePost)

# define total acquisition time to align pixels
AcqTime = np.mean(tot_acq_times)
NumSpePerLine = int(round(NumSpePerLine_unaligned.mean().item()))
TimeStamps_aligned = np.linspace(0, AcqTime, NumSpePerLine)

           # of spectra
# of line              
0                   166
1                   166
2                   167
3                   167
4                   167
5                   167
6                   167

line scan spectra summary
# of lines is: 7
mean # of spectra is: 167
min # of spectra is: 166


### peak intensity extraction for all pixels (with time stamp alignment)

In [4]:
# pixel intensity extraction
pixels = get_pixels_values_nofilter(NameBody, NamePost, LineList, NumSpePerLine_unaligned, TimeStamps_aligned, MassList, MassTolerance)
pixels = np.reshape(pixels, (len(LineList), int(pixels.shape[0]/len(LineList)), pixels.shape[1]))

line 0 is done, running time is:
4.73 s
 
line 1 is done, running time is:
4.35 s
 
line 2 is done, running time is:
4.41 s
 
line 3 is done, running time is:
4.32 s
 
line 4 is done, running time is:
4.89 s
 
line 5 is done, running time is:
4.94 s
 
line 6 is done, running time is:
4.55 s
 


In [17]:
#===========================================
# save/load file
#===========================================
## save file
Save_dir = MSI_data_output+'\\pixels.npy'
np.save(Save_dir, pixels)

## load file
#pixels = np.load(MSI_data_output+'\\pixels.npy')
#pixels.shape

In [5]:
# determine aspect ratio of image
width = 7*1000 # um
height = 7.5*1000 # um

aspect_ratio = (height/(pixels.shape[0]-1))/(width/(pixels.shape[1]-1))
print(aspect_ratio)

29.642857142857142


### generate TIC image

In [6]:
pixels_TIC = pixels[:,:,0]
pixels_feature = pixels[:,:,1:]
pixels_feature_TICnorm = pixels_feature/pixels_TIC[:,:,None]

In [ ]:
# Plot and/or save images:
# arguments are: 
#   pixels_feature = The array of images of shape (lines, scans, m/z). Edit this argument to be the correct array
#   MassList: No need to edit
#   includes_TIC: True if the first image of the array is the TIC
#   scale: scale pixel intensities to this quantile                
#   aspect: no need to edit
#   cmap: the colormap to use for display
#   savefig: True to save the images
#   savedir: The directory to save the images into
show_ms1_imgs(pixels_feature, MassList, includes_TIC = False, scale = .999, aspect = aspect_ratio, cmap = 'viridis', savefig = True, savedir = MSI_data_output)

### internal standard normalization

In [11]:
# the 0-indexed location of your internal standard in your MassList.
intl_std_index = 1

pixels_std = pixels_feature[:,:,intl_std_index].copy()
pixels_stdnorm = pixels_feature/pixels_std[:,:,None]

In [ ]:
show_ms1_imgs(pixels_feature, MassList, includes_TIC = False, scale = .999, aspect = aspect_ratio, cmap = 'viridis', savefig = False, savedir = MSI_data_output)